# 一般化した数独の問題作成&ソルバー by pulp
- 数独はニコリの登録商標です
- ナンプレとも呼ばれます
- pulpを使って整数計画問題を解く練習のために作ったnotebookです

## ソルバー
- 与えられた問題の解を全て出力する

## 問題作成
- 方法1: 空の盤面にランダムに数字を入れていき, ソルバーで解が一意かどうか判定しながらやる
- 方法2: 埋まった盤面からランダムに数字を抜いていき, ソルバーで解が一意かどうか判定しながらやる
    - 埋まった盤面を作る必要がある

## 一般化
- 普通の数独(9x9盤面)の正方形ブロック(3x3)を$m \times n$にした
- 盤面のサイズは $mn \times mn$

Prologバージョン
- https://github.com/birdwatcherYT/SudokuProlog/blob/master/sudoku.pro


## 整数計画問題
### pulp
インストール方法
- Anacondaの場合
    - `conda install -c conda-forge pulp`
- pipの場合
    - `pip install pulp`

使い方:
- https://docs.pyq.jp/python/math_opt/pulp.html

In [1]:
import numpy as np
import pulp

In [2]:
class Sudoku:
    """数独クラス"""
    def __init__(self, m, n):
        """考える小さいブロックのサイズ(m, n): 普通の数独ならm=3, n=3"""
        self.m = m
        self.n = n
        # 入る数字の最大数
        self.max_num = m * n
        # ループ用
        self.r = range(self.max_num)

    def init(self, problem):
        """与えられた問題を解くための初期化用関数
         problem: 0が空欄をあらわすnp.ndarray(int)
        """
        # 最適化問題
        self.sudoku = pulp.LpProblem("sudoku", pulp.LpMinimize)
        # max_num x max_numの盤面の数を表す変数
        self.board = np.array(
            [
                [
                    [
                        pulp.LpVariable(f"board_{i}_{j}_{k}", cat="Binary")
                        for k in self.r
                    ]
                    for j in self.r
                ]
                for i in self.r
            ]
        )
        # 数字はどれか一つ
        for i in self.r:
            for j in self.r:
                self.sudoku += pulp.lpSum(self.board[i,j,:]) == 1
        # 行に同じ数字が入ってはいけない
        for i in self.r:
            for k in self.r:
                self.sudoku += pulp.lpSum(self.board[i,:,k]) == 1
        # 列に同じ数字が入ってはいけない
        for j in self.r:
            for k in self.r:
                self.sudoku += pulp.lpSum(self.board[:,j,k]) == 1
        # 正方形の枠に同じ数字が入ってはいけない
        for i in range(0, self.max_num, self.m):
            for j in range(0, self.max_num, self.n):
                for k in self.r:
                    self.sudoku += pulp.lpSum(self.board[i:i+self.m, j:j+self.n, k]) == 1
        # 初期値をいれる
        for i in self.r:
            for j in self.r:
                if problem[i, j]:# 空欄でないとき
                    self.sudoku += self.board[i, j, problem[i, j]-1] == 1

    def solve(self, problem):
        """ソルバー. 全ての解が順番に得られる
         problem: 0が空欄をあらわすnp.ndarray(int)
        """
        self.init(problem)
        while True:
            self.sudoku.solve()
            if pulp.LpStatus[self.sudoku.status] != "Optimal":
                break
            answer = np.vectorize(pulp.value)(self.board)
            answer = np.where(answer)[2].reshape(self.max_num,self.max_num)+1
            yield answer
            # 割り当てられた変数から最低でも1つ外すようにすることで別の解を探すようにする
            self.sudoku += pulp.lpSum([
                self.board[i, j, k]
                for i in self.r
                for j in self.r
                for k in self.r
                if pulp.value(self.board[i, j, k]) == 1
            ]) <= self.max_num * self.max_num-1        
    
    def is_unique(self, problem):
        """解が一意かどうか判定
         True: 一意
         False: 一意でない
         None: 解なし
        """
        cnt = 0
        for ans in self.solve(problem):
            cnt+=1
            # 一意でない
            if cnt>=2:
                return False
        # 一意
        if cnt==1:
            return True
        # 解なし
        return None

    def create_problem_from_empty(self, try_num):
        """問題作成 (空の盤面にランダムに数字を入れていくバージョン)
         実行速度遅い.
        """
        # 空の盤面作成
        problem = np.zeros((self.max_num, self.max_num), dtype=int)
        # 埋めていく
        for _ in range(try_num):
            print(">", end="")
            while True:
                i, j = np.random.randint(self.max_num, size=2)
                if not problem[i, j]:
                    break
            # 盤面(i,j)を適当に埋める
            problem[i, j] = np.random.randint(self.max_num) + 1
            is_unique = self.is_unique(problem)
            # 一意でないなら盤面を空白に戻す
            if is_unique is None:
                problem[i, j] = 0
                continue
            # 一意なら終了
            if is_unique:
                print("")
                return problem
        return None

    def create_problem_from_fill(self, try_num, level=1):
        """問題作成 (埋まった状態から穴をあけていくバージョン)
         最初に答えの盤面を作る必要がある. 
         try_num: 答えの盤面作成で試行する回数. m*nくらいが良さそう. 
         level: 穴あけ失敗許容回数
        """
        # 空の盤面作成
        problem = np.zeros((self.max_num, self.max_num), dtype=int)
        # 盤面を全て埋める
        if not self.__fill_rows(problem, 0, try_num):
#         if not fill_rows(self.m, self.n, self.max_num, problem, 0, try_num):
            return None

        cnt = 0 # 穴あけに失敗した回数
        while True:
            print(">", end="")
            while True:
                i, j = np.random.randint(self.max_num, size=2)
                if problem[i, j]:
                    break
            tmp = problem[i, j]
            # 盤面に穴をあける
            problem[i, j] = 0
            if not self.is_unique(problem):
                # 一意でなくなったら値をもとに戻す
                problem[i, j] = tmp
                cnt += 1
                if cnt >= level:
                    break
        print("")
        return problem

    def print_board(self, problem):
        """盤面の表示"""
        for i in self.r:
            if i%self.m==0:
                print("----"*self.max_num+"-"*(self.m+1))
            for j in self.r:
                if j%self.n==0:
                    print("|", end="")
                print(f"{problem[i,j]:>3} " if problem[i,j] else "    ", end="")
            print("|")
        print("----"*self.max_num+"-"*(self.m+1))

    def __fill_rows(self, problem, i, try_num):
        """i行以降をランダムに埋める再帰関数 (埋まった状態の盤面作成用ヒューリスティック)"""
        if i >= self.max_num:
            return True
        for _ in range(try_num):
            if not self.__fill_vals(problem, i, 0, try_num):
                continue
            if self.__fill_rows(problem, i+1, try_num):
                return True
        return False

    def __fill_vals(self, problem, i, j, try_num):
        """i行目のj列以降をランダムに埋める再帰関数 (埋まった状態の盤面作成用ヒューリスティック)"""
        if j >= self.max_num: # 最後の列まで埋まったら
            return True
        for _ in range(try_num):
            problem[i, j] = np.random.randint(self.max_num)+1
            if self.check_duplicate(problem, i, j):
                continue # 重複したら別の数字をトライ
            if self.__fill_vals(problem, i, j+1, try_num): #次の値を埋めに行く
                return True
        problem[i, j] = 0
        return False

    def check_duplicate(self, problem, i, j):
        """(i,j)が属する行、列、ブロックで、0以外の数字が重複してたらTrue"""
        row = problem[i, :]
        zero_row = (row==0).any()
        col = problem[:, j]
        zero_col = (col==0).any()
        blk = problem[i//self.m*self.m:(i//self.m+1)*self.m, j//self.n*self.n:(j//self.n+1)*self.n]
        zero_blk = (blk == 0).any()
        return (np.unique(row).shape[0]-zero_row != (row!=0).sum() or \
            np.unique(col).shape[0]-zero_col != (col!=0).sum() or \
            np.unique(blk).shape[0]-zero_blk != (blk != 0).sum()
        )

    def __solve_dfs(self, problem, pos):
        """おまけ：深さ優先探索で解を求める"""
        i, j = pos // self.max_num, pos % self.max_num
        if pos >= self.max_num*self.max_num:
            # すべて埋まったら
            yield problem
            return
        if problem[i, j] != 0: # 数字が埋まっていれば次のマスへ
            yield from self.__solve_dfs(problem, pos+1)
            return
        # すべての数字を試す
        for k in self.r:
            problem[i, j] = k+1
            if self.check_duplicate(problem, i, j):
                continue # 重複したら次の数字をトライ
            yield from self.__solve_dfs(problem, pos+1)
        problem[i, j] = 0

    def solve_dfs(self, problem):
        """おまけ：深さ優先探索で解を求める"""
        return self.__solve_dfs(problem.copy(), 0)


In [3]:
# m, n = 1, 2
# m, n = 2, 2
# m, n = 2, 3
m, n = 3, 3
# m, n = 2, 5
# m, n = 2, 6
# m, n = 3, 4
# m, n = 4, 4
# m, n = 4, 5

sudoku = Sudoku(m, n)

## 埋まった盤面から穴を開けていく方法

In [4]:
%%time
problem = sudoku.create_problem_from_fill(m*n, level=1)
sudoku.print_board(problem)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
----------------------------------------
|      6   9 |  7         |      1     |
|  4       8 |  5   3     |      9     |
|  5   7   3 |          6 |  4   2     |
----------------------------------------
|  1   2     |  4         |            |
|            |      2     |  3   4   7 |
|      3   4 |      8   5 |      6     |
----------------------------------------
|  8   4     |            |  6   3   9 |
|  3   9     |  8         |  2   5     |
|  6       2 |            |  8         |
----------------------------------------
CPU times: user 1.94 s, sys: 422 ms, total: 2.37 s
Wall time: 5.41 s


In [5]:
%%time
# 作った問題を解く
for ans in sudoku.solve(problem):
    sudoku.print_board(ans)

----------------------------------------
|  2   6   9 |  7   4   8 |  5   1   3 |
|  4   1   8 |  5   3   2 |  7   9   6 |
|  5   7   3 |  1   9   6 |  4   2   8 |
----------------------------------------
|  1   2   6 |  4   7   3 |  9   8   5 |
|  9   8   5 |  6   2   1 |  3   4   7 |
|  7   3   4 |  9   8   5 |  1   6   2 |
----------------------------------------
|  8   4   1 |  2   5   7 |  6   3   9 |
|  3   9   7 |  8   6   4 |  2   5   1 |
|  6   5   2 |  3   1   9 |  8   7   4 |
----------------------------------------
CPU times: user 64.3 ms, sys: 19.1 ms, total: 83.5 ms
Wall time: 141 ms


In [6]:
%%time
# おまけ：深さ優先探索で問題を解く
for ans in sudoku.solve_dfs(problem):
    sudoku.print_board(ans)

----------------------------------------
|  2   6   9 |  7   4   8 |  5   1   3 |
|  4   1   8 |  5   3   2 |  7   9   6 |
|  5   7   3 |  1   9   6 |  4   2   8 |
----------------------------------------
|  1   2   6 |  4   7   3 |  9   8   5 |
|  9   8   5 |  6   2   1 |  3   4   7 |
|  7   3   4 |  9   8   5 |  1   6   2 |
----------------------------------------
|  8   4   1 |  2   5   7 |  6   3   9 |
|  3   9   7 |  8   6   4 |  2   5   1 |
|  6   5   2 |  3   1   9 |  8   7   4 |
----------------------------------------
CPU times: user 56.3 ms, sys: 14.2 ms, total: 70.5 ms
Wall time: 59.1 ms


## 空欄の盤面を埋めていく方法
$mn>10$ からきつくなる

In [7]:
%%time
problem = sudoku.create_problem_from_empty(1000)
sudoku.print_board(problem)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
----------------------------------------
|  4         |  8         |      1     |
|      2   7 |      5     |          8 |
|            |  2   1     |  7   6     |
----------------------------------------
|          8 |  9       1 |            |
|  6   7     |  3         |  8   2   9 |
|  9   5   2 |  7         |      3   4 |
----------------------------------------
|      6   4 |  1         |          2 |
|      9     |  4       8 |          1 |
|            |      7     |  4       6 |
----------------------------------------
CPU times: user 3.81 s, sys: 790 ms, total: 4.6 s
Wall time: 12.1 s


In [8]:
%%time
# 作った問題を解く
for ans in sudoku.solve(problem):
    sudoku.print_board(ans)

----------------------------------------
|  4   3   6 |  8   9   7 |  2   1   5 |
|  1   2   7 |  6   5   3 |  9   4   8 |
|  5   8   9 |  2   1   4 |  7   6   3 |
----------------------------------------
|  3   4   8 |  9   2   1 |  6   5   7 |
|  6   7   1 |  3   4   5 |  8   2   9 |
|  9   5   2 |  7   8   6 |  1   3   4 |
----------------------------------------
|  7   6   4 |  1   3   9 |  5   8   2 |
|  2   9   5 |  4   6   8 |  3   7   1 |
|  8   1   3 |  5   7   2 |  4   9   6 |
----------------------------------------
CPU times: user 77.3 ms, sys: 19.5 ms, total: 96.7 ms
Wall time: 144 ms


# おまけ
- 盤面埋めるプロセスをjitで高速化したバージョン
- 実装が汚くなるので一旦ここまで避けてある (コメントアウトして使ってない)

In [9]:
from numba import njit

@njit
def fill_rows(m,n,max_num, problem, i, try_num):
    if i >= max_num:
        return True
    for _ in range(try_num):
        if not fill_vals(m,n,max_num,problem, i, 0, try_num):
            continue
        if fill_rows(m,n,max_num,problem, i+1, try_num):
            return True
    return False
@njit
def fill_vals(m,n,max_num, problem, i, j, try_num):
    if j >= max_num:
        return True
    for _ in range(try_num):
        problem[i,j] = np.random.randint(max_num)+1
        if check_duplicate(m,n,problem, i, j):
            continue
        if fill_vals(m,n,max_num,problem, i, j+1, try_num):
            return True
    problem[i,j] = 0
    return False
@njit
def check_duplicate(m,n, problem, i, j):
    """(i,j)が属する行、列、ブロックで、0以外の数字が重複してたらTrue"""
    row = problem[i, :]
    zero_row = (row==0).any()
    col = problem[:, j]
    zero_col = (col==0).any()
    blk = problem[i//m*m:(i//m+1)*m, j//n*n:(j//n+1)*n]
    zero_blk = (blk == 0).any()
    return (np.unique(row).shape[0]-zero_row != (row!=0).sum() or \
        np.unique(col).shape[0]-zero_col != (col!=0).sum() or \
        np.unique(blk).shape[0]-zero_blk != (blk != 0).sum()
    )
